# Interview Cake SQL Questions

### Introduction

Interview Cake's sample SQL interview questions. Uses MySQL.

Create an anconda environment based off enivroment.yml and acticate it with these commands:
```bash
conda env create -f environment.yml -n conda env update -f environment.yml -n sql-query-practice
source activate sql-query-practice
```
**NOTE**: You may need to add the environment to Jupyter with this command

```bash
conda env update -f environment.yml -n sql-query-practice
```


In your terminal, download the script and start up MySQL:

```bash
curl -O https://static.interviewcake.com/bakery_schema_and_data.sql && mysql.server start && mysql -u root
```

We'll be going through six questions covering topics like query performance, joins, and SQL injection. They'll refer to the same database for cakes, customers, and orders at a bakery. Here's the schema:

```sql
CREATE TABLE cakes (
    cake_id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    flavor VARCHAR(100) NOT NULL
);

CREATE TABLE customers (
    customer_id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    first_name VARCHAR(100) NOT NULL,
    last_name VARCHAR(100) NOT NULL,
    phone VARCHAR(15),
    street_address VARCHAR(255),
    city VARCHAR(255),
    zip_code VARCHAR(5),
    referrer_id INT,
    FOREIGN KEY (referrer_id) REFERENCES customers (customer_id)
);

CREATE TABLE orders (
    order_id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    cake_id INT NOT NULL,
    customer_id INT,
    pickup_date DATE NOT NULL,
    FOREIGN KEY (cake_id) REFERENCES cakes (cake_id),
    FOREIGN KEY (customer_id) REFERENCES customers (customer_id)
);
```

In [1]:
# Loads watermark extension and prints details about current platform
%load_ext watermark
%watermark -v -n -m -p numpy,scipy,sklearn,pandas,pymysql
 
# import packages
%matplotlib inline
%load_ext dotenv
# Use find_dotenv to locate the file
%dotenv

import pandas as pd
import pandas.io.sql as mysql
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pymysql

PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
print(PROJ_ROOT)
import sys
sys.path.append(os.path.join(PROJ_ROOT, 'src'))

Thu Nov 22 2018 

CPython 3.7.0
IPython 7.1.1

numpy 1.15.4
scipy 1.1.0
sklearn 0.20.0
pandas 0.23.4
pymysql 0.9.2

compiler   : Clang 4.0.1 (tags/RELEASE_401/final)
system     : Darwin
release    : 17.5.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit
/Users/sebp/LocalDocuments2/DataScience/portfolio/SQL/interview_cake_sql


### 0 - Initialize Database and Preview Data

The bash script from earlier downloaded the data and imported it into a Postgres database. Now it's time to import the data into a pandas dataframe in this notebook.

In [2]:
connection = pymysql.connect(user = os.getenv("DB_USER"),
                                  password = os.getenv("DB_PASSWORD"),
                                  host = os.getenv("DB_HOST"),
                                  database = os.getenv("DB_NAME"))


cur = connection.cursor()

cakes_df = mysql.read_sql("Select * from cakes", connection)
customers_df = mysql.read_sql("Select * from customers", connection)
orders_df = mysql.read_sql("Select * from orders", connection)

Let's preview each table.

In [3]:
print ('Cakes Table')
cakes_df.head()

Cakes Table


,cake_id,flavor
0,1,Chocolate
1,2,Red Velvet
2,3,White
3,4,Carrot


In [4]:
print ('Customers Table')
customers_df.head()

Customers Table


,customer_id,first_name,last_name,phone,street_address,city,zip_code,referrer_id
0,1,Tim,Mccain,8015550120,7527 Holly Brooks,Springfield,33218,NaN
1,2,James,White,8015550117,61476 Raynor Points Apt. 077,Springfield,33218,NaN
2,3,Wendy,Morris,8015550173,51897 Hills Passage Apt. 398,Springfield,33218,NaN
3,4,Mary,Wagner,8345550194,540 Martell Point,Dover,33220,NaN
4,5,Mattie,Rainey,8015550112,348 Milburn Island,Springfield,33218,3.0


In [12]:
print ('Orders Table')
orders_df.head()

Orders Table


,order_id,cake_id,customer_id,pickup_date
0,1,2,NaN,2017-01-08
1,2,1,NaN,2017-09-17
2,3,3,NaN,2017-07-19
3,4,2,NaN,2017-12-03
4,5,2,NaN,2017-08-20


# Questions

### 1. - How can we make this query faster?

We want the order ID of every order in January and February, 2017. This is the query we've got so far:

```sql
SELECT order_id FROM orders WHERE DATEDIFF(orders.pickup_date, '2017-03-01') < 0;
-- 161314 rows in set (0.25 sec)
```

Let's time the above query to find out what our current run time is.

In [6]:
%time

query = '''
SELECT order_id 
FROM orders 
WHERE DATEDIFF(orders.pickup_date, '2017-03-01') < 0;
'''


orders_jan_feb_2017 = mysql.read_sql(query, connection)


orders_jan_feb_2017.head()

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 12.2 µs


,order_id
0,1
1,14
2,21
3,26
4,32


The easy answer is adding an index. Adding an index makes inserts slower, querying much faster. MySQL allows for two types of indexes: B-Tree and Hash indexes.

- A B-tree index can be used for column comparisons in expressions that use the =, >, >=, <, <=, or BETWEEN operators
- A Hash index is used only for equality comparisons that use the = or <=> operators (but are very fast). They are not used for comparison operators such as < that find a range of values. The optimizer cannot use a hash index to speed up ORDER BY operations.

To add an index, use one of the following MySQL commands:

```sql
create index your_index_name on your_table_name(your_column_name) using HASH;
or
create index your_index_name on your_table_name(your_column_name) using BTREE;
```

Because we are searching for dates in between two values, we want a Hash index.

**NOTE**: If an index has already been added, run the commented out code to remove it.

In [7]:
# query= '''
# ALTER TABLE orders DROP INDEX IF EXISTS pickup_date_index;
# '''
# cur.execute(query)

query= '''
create index pickup_date_index on orders(pickup_date) using HASH;
'''

cur.execute(query)

/Users/sebp/miniconda3/envs/sql-query-practice/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (3502, 'This storage engine does not support the HASH index algorithm, storage engine default was used instead.')
  result = self._query(query)


0

In [8]:
%time

query = '''
SELECT order_id 
FROM orders 
WHERE orders.pickup_date < '2017-03-01';
'''


orders_jan_feb_2017_improved = mysql.read_sql(query, connection)


orders_jan_feb_2017_improved.head()

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 10 µs


,order_id
0,150
1,553
2,969
3,1575
4,1857


### 2. How can we get the nth highest number of orders a single customer has?

**SOLUTION**:

Do a join on the customers table and the orders table. On the orders table, count how many orders belong to each customer. Join On customer_id's.

Use the Limit clause to select the nth highest order. If LIMIT has one argument, that argument is the number of rows to return starting with the first row. With 2 arguments, the first argument is the row offset and the second argument is the number of rows to return. So in our query with “1, 1” we're saying "starting one row down from the top, give us one row."

We subtract the nth_clause because as stated above, 2 arguments will automatically offset the starting row by 1, and we want to offset by the inputted nth_highest, not nth_highest + 1.

In [31]:
%time

query = '''
SELECT a.customer_id, a.first_name, b.order_count 
FROM customers a JOIN (
    SELECT customer_id, COUNT(DISTINCT order_id) as order_count
    FROM orders
    GROUP BY customer_id
    ) b
ON a.customer_id = b.customer_id
ORDER BY order_count DESC
LIMIT %d, 1;
'''

nth_highest = 6


nth_highest_num_orders = mysql.read_sql(query  % (nth_highest-1), connection)


nth_highest_num_orders.head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


,customer_id,first_name,order_count
0,773260,Cecile,4


### 3. What ways can we use wildcard characters in LIKE clauses?

**ANSWER**:

Wildcard characters are used to substitute other characters in a string. They are used in conjunction with the LIKE clause to find values's that partially match the inputted string.


There are two wildcard characters:

 - % - The percent sign represents zero, one, or multiple characters
 - _ -The underscore represents a single character
 
For example, a% finds any values that start with 'a'.

For example, _a finds any values that start that have 'a' as the second character.

### 4. How can we make this query faster?
```sql
SELECT first_name, last_name, street_address, city, zip_code FROM customers
WHERE first_name LIKE '%sam%' AND city = 'Dover';
-- 1072 rows in set (0.42 sec)
```

Let's first run this query locally to see how fast it runs.

In [37]:
%time

query = '''
SELECT first_name, last_name, street_address, city, zip_code 
FROM customers
WHERE first_name LIKE '%sam%' AND city = 'Dover'
'''


slow_query = mysql.read_sql(query, connection)


slow_query.head()

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 11 µs


,first_name,last_name,street_address,city,zip_code
0,Samuel,Castillo,24820 Krajcik Fields,Dover,33220
1,Samantha,Averette,94349 Sauer Isle Suite 427,Dover,33220
2,Samantha,Jones,4343 Shirley Lane Apt. 915,Dover,33220
3,Samantha,Caballero,3414 Paola Drive Apt. 744,Dover,33220
4,Samantha,Hayes,6037 Tinnie Drive,Dover,33220


**SOLUTION**

MySQL can't uses indices on queries that start with a wildcard. Lets start by removing the leading wildcard character in `%sam%`.

In [38]:
%time

query = '''
SELECT first_name, last_name, street_address, city, zip_code 
FROM customers
WHERE first_name LIKE 'sam%' AND city = 'Dover'
'''


faster_query = mysql.read_sql(query, connection)


faster_query.head()

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.91 µs


,first_name,last_name,street_address,city,zip_code
0,Samuel,Castillo,24820 Krajcik Fields,Dover,33220
1,Samantha,Averette,94349 Sauer Isle Suite 427,Dover,33220
2,Samantha,Jones,4343 Shirley Lane Apt. 915,Dover,33220
3,Samantha,Caballero,3414 Paola Drive Apt. 744,Dover,33220
4,Samantha,Hayes,6037 Tinnie Drive,Dover,33220


Next, we can add an index to the `city` column. We will use MySQL's default index selection this time.

In [39]:
query= '''
ALTER TABLE customers ADD INDEX (city);
'''

cur.execute(query)

0

And let's run the query again.

In [41]:
%time

query = '''
SELECT first_name, last_name, street_address, city, zip_code 
FROM customers
WHERE first_name LIKE 'sam%' AND city = 'Dover'
'''


faster_query = mysql.read_sql(query, connection)


faster_query.head()

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs


,first_name,last_name,street_address,city,zip_code
0,Samuel,Castillo,24820 Krajcik Fields,Dover,33220
1,Samantha,Averette,94349 Sauer Isle Suite 427,Dover,33220
2,Samantha,Jones,4343 Shirley Lane Apt. 915,Dover,33220
3,Samantha,Caballero,3414 Paola Drive Apt. 744,Dover,33220
4,Samantha,Hayes,6037 Tinnie Drive,Dover,33220


Finally, lets add a index to `first_name`.

In [42]:
query= '''
ALTER TABLE customers ADD INDEX (first_name);
'''

cur.execute(query)

%time

query = '''
SELECT first_name, last_name, street_address, city, zip_code 
FROM customers
WHERE first_name LIKE 'sam%' AND city = 'Dover'
'''


fastest_query = mysql.read_sql(query, connection)


fastest_query.head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


,first_name,last_name,street_address,city,zip_code
0,Sam,Silva,5420 Thurston Expressway,Dover,33220
1,Sam,Seal,7060 Boehm Valleys,Dover,33220
2,Sam,Davis,60235 Angel Underpass Apt. 626,Dover,33220
3,Sam,Digeorgio,88601 Mae Mill Apt. 152,Dover,33220
4,Sam,Olson,150 Rylie Plain Suite 549,Dover,33220


**Possible Query Changes**

The Interview Cake solution suggests removing `city` and `zipcode` from our query since we already know that the city we are requesting in our query is Dover, and the zipcode for Dover is 33220. I decided against this change in order to keep the query as similar to the original as possible.

With that said, removing `city` and `zipcode` is a valid way to speed up the query.

### 5. What are all the SQL joins?

**ANSWER**

